In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import win32com.client as win32
import sqlite3 as sql
from scipy.optimize import minimize
from scipy.optimize import fsolve
import time

In [133]:
aspen=win32.Dispatch('Apwn.Document')
direccion="C:\\Users\\ASUS\\Documents\\Universidad\\Tesis\\Convergencia_Newton\\Simulación\\equation_oriented.apw"

# def doble_line(dir):
#     return s.replace("\\", "\\\\")

# direccion=doble_line(direccion)
aspen.InitFromArchive2(direccion)
aspen.visible=False
print(f"Aspen cargado")
def correr_simulacion(aspen):
    aspen.Engine.Run2()
    while aspen.Engine.isRunning:
        time.sleep(1)



Aspen cargado


In [134]:
def error(x1,x2):
    return abs(x1-x2)/x1
def calculo_error(x,args):
    aspen=args
    flujo,x1,x2=x
    x3=1-x1-x2
    if x1+x2>=1:
        x1=x1/(x1+x2)
        x2=x2/(x1+x2)
        x3=0
    if x1>=1:
        x1=1
        x2=0
        x3=0
    if x2<=0:
        x2=0
    bloques=aspen.Tree.Elements.Item('Data').Elements.Item('Blocks')
    corrientes=aspen.Tree.Elements.Item('Data').Elements.Item('Streams')
    corriente_ingreso=corrientes.Elements.Item('GLR+').Elements.Item("Input")
    Flujo_ingreso=corriente_ingreso.Elements.Item("TOTFLOW").Elements.Item("MIXED")
    Flujo_ingreso.Value=flujo
    concentracion_ingreso=corriente_ingreso.Elements.Item("FLOW").Elements.Item("MIXED")
        
    concentracion_ingreso.Elements.Item("GLYCE-01").Value=x1
    concentracion_ingreso.Elements.Item("WATER").Value=x2
    concentracion_ingreso.Elements.Item("ETHAN-01").Value=x3
    
    correr_simulacion(aspen)
    
    corriente_salida=corrientes.Elements.Item('GLR').Elements.Item("Output")
    flujo_salida=corriente_salida.Elements.Item("TOT_FLOW").Value
    concentracion_salida=corriente_salida.Elements.Item("MOLEFRAC").Elements.Item("MIXED")
    salida_glicerina=concentracion_salida.Elements.Item("GLYCE-01").Value
    salida_agua=concentracion_salida.Elements.Item("WATER").Value
    salida_etanol=concentracion_salida.Elements.Item("ETHAN-01").Value
    error_glicerina=error(salida_glicerina,x1)
    error_agua=error(salida_agua,1-x1)
    error_flujo=error(flujo_salida,flujo)
    error_etanol=error(salida_etanol,1-x1-x2)
    return [error_flujo,error_glicerina,error_agua]
    # norma=np.linalg.norm([error_glicerina,error_agua,error_flujo,error_etanol])
    # return norma
    
    

In [135]:
def jacobiano(variables,ASPEN):
    flujo,x1,x2=variables
    delta=0.001
    flujo_1=flujo+delta*flujo
    flujo_2=flujo-delta*flujo
    x1_1=x1+delta*x1
    x1_2=x1-delta*x1
    x2_1=x2+delta*x2
    x2_2=x2-delta*x2
    if x1_1>=1:
        x1_1=1
    if x1>1:
        x1=1
    if x2_1<0:
        x2_1=0
    delta_flujo=(flujo_1-flujo_2)
    delta_x1=x1_1-x1_2
    delta_x2=x2_1-x2_2
    J=np.zeros((3,3))
    J[0,0]=(calculo_error([flujo_1,x1,x2],ASPEN)[0]-calculo_error([flujo_2,x1,x2],ASPEN)[0])/delta_flujo
    J[0,1]=(calculo_error([flujo,x1_1,x2],ASPEN)[0]-calculo_error([flujo,x1_2,x2],ASPEN)[0])/delta_x1
    J[0,2]=(calculo_error([flujo,x1,x2_1],ASPEN)[0]-calculo_error([flujo,x1,x2_2],ASPEN)[0])/delta_x2
    J[1,0]=(calculo_error([flujo_1,x1,x2],ASPEN)[1]-calculo_error([flujo_2,x1,x2],ASPEN)[1])/delta_flujo
    J[1,1]=(calculo_error([flujo,x1_1,x2],ASPEN)[1]-calculo_error([flujo,x1_2,x2],ASPEN)[1])/delta_x1
    J[1,2]=(calculo_error([flujo,x1,x2_1],ASPEN)[1]-calculo_error([flujo,x1,x2_2],ASPEN)[1])/delta_x2
    J[2,0]=(calculo_error([flujo_1,x1,x2],ASPEN)[2]-calculo_error([flujo_2,x1,x2],ASPEN)[2])/delta_flujo
    J[2,1]=(calculo_error([flujo,x1_1,x2],ASPEN)[2]-calculo_error([flujo,x1_2,x2],ASPEN)[2])/delta_x1
    J[2,2]=(calculo_error([flujo,x1,x2_1],ASPEN)[2]-calculo_error([flujo,x1,x2_2],ASPEN)[2])/delta_x2
    return J
    

In [136]:
def newton_raphson(f,variables,ASPEN,iteraciones=10):
    tol=1e-6
    error=1
    iteracion=0
    memoria=np.zeros((0,3))
    memoria=np.append(memoria,[variables],axis=0)
    while error>tol and iteracion<iteraciones:
        J=jacobiano(variables,ASPEN)
        print(J)
        F=np.array(f(variables,ASPEN))
        delta=np.linalg.inv(J).dot(-F)
        # delta=np.linalg.solve(J,-F)
        variables=variables+delta
        error=np.linalg.norm(F)
        memoria=np.append(memoria,[variables],axis=0)
        print(iteracion)
        iteracion+=1
    return variables,memoria,iteracion,F

In [137]:
x=[99,0.9997,0.00029]
calculo_error(x,aspen)

[0.003516249984250415, 0.0002995251422744098, 630.5470070446668]

In [138]:
jacobiano(x,aspen)

array([[-3.56426015e-05,  0.00000000e+00,  0.00000000e+00],
       [-2.52449739e-08, -7.76928958e-01, -1.72362228e-03],
       [-3.02819310e+01, -1.59705355e+06, -9.10843233e+05]])

In [139]:
newton_raphson(calculo_error,x,aspen,iteraciones=10)

[[-3.56426015e-05  0.00000000e+00  0.00000000e+00]
 [-2.52449739e-08 -7.76928958e-01 -1.72362228e-03]
 [-3.02819310e+01 -1.59705355e+06 -9.10843233e+05]]
0
[[-3.49484563e-05  0.00000000e+00  0.00000000e+00]
 [-5.71291001e-05 -1.17741648e+00  0.00000000e+00]
 [ 0.00000000e+00  8.82542677e+01  0.00000000e+00]]


LinAlgError: Singular matrix

In [141]:
aspen.Close()
aspen.Quit()

com_error: (-2147417848, 'El objeto invocado ha desconectado de sus clientes.', None, None)